In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### 통계기반 기법의 문제
주변 단어의 빈도를 기초로 단어를 표현 그러나 현업에서 다루는 말뭉치의 어휘 수는 많다
<br/>

### 통계 기반과 추론 기반 차이
통계 기반 기법 : 말뭉치 전체의 통계(동시 발생 행렬과 PPMI 등)을 이용해 단 1회 처리만의 단어 분산 표현
추론 기반 기법 : 미니 배치의 학습 샘플씩 반복하여 가중치를 갱신해간다

### 추론 기반 기법 
맥락을 입력하면 모델은 각 단어의 출현 확률을 출력한다
모델이 올바른 추측을 내놓도록 학습 시키고 그 결과로 단어의 분산 표현을 얻는 것

In [1]:
import numpy as np

c = np.array([[1, 0, 0, 0, 0, 0, 0]])
W = np.random.randn(7,3)
h = np.matmul(c,W)
print(h)

[[-0.49567883  0.6918531   0.91776982]]


In [6]:
import sys
sys.path.append('..')
import numpy as np
from common.layers import MatMul
c = np.array([[1, 0, 0, 0, 0, 0, 0]])
W = np.random.randn(7,3)
layer = MatMul(W)
h = layer.forward(c)
print(h)

[[-0.47791988 -1.00990464  1.28103816]]


### CBOW 
맥락(주변단어)으로부터 중앙단어을 추측하는 용도의 신경망 
<b>은닉층의 뉴런 수를 입력층의 뉴런 수보다 적게 하는 것이 중요한 핵심</b>

In [7]:
# coding: utf-8
import sys
sys.path.append('..')
import numpy as np
from common.layers import MatMul


# 샘플 맥락 데이터
c0 = np.array([[1, 0, 0, 0, 0, 0, 0]])
c1 = np.array([[0, 0, 1, 0, 0, 0, 0]])

# 가중치 초기화
W_in = np.random.randn(7, 3)
W_out = np.random.randn(3, 7)

# 계층 생성
in_layer0 = MatMul(W_in)
in_layer1 = MatMul(W_in)
out_layer = MatMul(W_out)

# 순전파
h0 = in_layer0.forward(c0)
h1 = in_layer1.forward(c1)
h = 0.5 * (h0 + h1)
s = out_layer.forward(h)
print(s)


[[-2.91523379 -0.06004395 -1.57696206  0.34791734  0.71327536  1.27387991
   0.48131375]]


In [8]:
import sys
sys.path.append('..')
from common.util import preprocess

text = "You say goodbye and I say hello"
corpus, word_to_id, id_to_word = preprocess(text)
print(corpus)

print(id_to_word)

[0 1 2 3 4 1 5]
{0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello'}


In [11]:
#타깃과 맥락을 만드는 함수
def create_contexts_target(corpus, window_size=1):
    target = corpus[window_size:-window_size]
    contexts = []
    
    for idx in range(window_size, len(corpus)-window_size):
        cs = []
        for t in range(-window_size, window_size+1):
            if t == 0:
                continue
            cs.append(corpus[idx + t])
        contexts.append(cs)
        
    return np.array(contexts), np.array(target)

In [13]:
import sys 
sys.path.append('..')
from common.util import preprocess, create_contexts_target, convert_one_hot

text = 'You say goodbye and I say hello'
corpus, word_to_id, id_to_word = preprocess(text)

contexts, target = create_contexts_target(corpus, window_size=1)

vocab_size = len(word_to_id)
target = convert_one_hot(target, vocab_size)
contexts = convert_one_hot(contexts, vocab_size)